In [1]:
import ee
import geemap

In [13]:
# 显示地图，并选中湖泊
Map = geemap.Map(center=[40.964,108.884],zoom = 10)
Map

Map(center=[40.964, 108.884], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [14]:
#在地图中选择研究区，并转为ROI。选定好后再运行下两行代码
Map.draw_last_feature
roi = ee.FeatureCollection(Map.draw_last_feature)

In [15]:
l8_vis_params = {
  'bands':['B5','B4','B3'],
  'min':0,
  'max': 0.3
}

In [17]:
# 选择L8-TOA影像
l8_col = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(roi)\
    .filter(ee.Filter.calendarRange(2019,2019,'year'))\
    .filter(ee.Filter.calendarRange(7,10,'month'))\

print(f"""已选择{l8_col.size().getInfo()}景L8影像""")
print(l8_col.aggregate_array('system:index').getInfo())
print(l8_col.aggregate_array('CLOUD_COVER').getInfo())

已选择22景L8影像
['LC08_128031_20190703', 'LC08_128031_20190804', 'LC08_128031_20190820', 'LC08_128031_20190905', 'LC08_128031_20190921', 'LC08_128031_20191007', 'LC08_128031_20191023', 'LC08_128032_20190703', 'LC08_128032_20190804', 'LC08_128032_20190820', 'LC08_128032_20190905', 'LC08_128032_20190921', 'LC08_128032_20191007', 'LC08_128032_20191023', 'LC08_129031_20190710', 'LC08_129031_20190726', 'LC08_129031_20190811', 'LC08_129031_20190827', 'LC08_129032_20190710', 'LC08_129032_20190726', 'LC08_129032_20190811', 'LC08_129032_20190827']
[0, 92.55000305175781, 9.15999984741211, 5.449999809265137, 13.65999984741211, 0.009999999776482582, 90.25, 0.41999998688697815, 75.20999908447266, 9.430000305175781, 17.030000686645508, 3.5299999713897705, 0.019999999552965164, 52.810001373291016, 6.260000228881836, 31.84000015258789, 18.56999969482422, 0.5699999928474426, 7.039999961853027, 16.209999084472656, 6.039999961853027, 0.8700000047683716]


In [11]:
def l8_toa_select_by_cloud(l8_col,roi,threshold):
    ##依次传入影像集合、研究区roi，云量阈值
    
    #计算l8_collection-Imagecollection中Image数量
    count = l8_col.size().getInfo()
    print(count)
    #定义图像裁剪函数
    def img_clip(image):
        clip_img = image.clip(roi)
        return clip_img        
    
    #进行图像裁剪
    l8_col_clip = l8_col.map(img_clip)
    l8_col_clip_list = l8_col_clip.toList(count)
    #创建列表，用来存储满足的影像名
    nice_list = []
    
    #定义云量计算函数
    def l8_img_cloud_cal(image):
        image = ee.Algorithms.Landsat.simpleCloudScore(image)
        cloud = image.select('cloud')
        hist = cloud.reduceRegion(ee.Reducer.histogram(100), roi, 30).getInfo()
        count_clear = sum(hist['cloud']['histogram'][0:10])
        count_sum = sum(hist['cloud']['histogram'])
        print(f"""
            {image.getInfo()['properties']['system:index']}研究区云量为：{1-(count_clear/count_sum)},全景云量为:{image.getInfo()['properties']['CLOUD_COVER']/100},
            被选中：{(1-(count_clear/count_sum))<threshold}""")
        if((1-(count_clear/count_sum))<threshold):
            nice_list.append('LANDSAT/LC08/C01/T1_TOA/'+image.getInfo()['properties']['system:index'])
    
    #遍历Image，计算云量，获取满足条件的影像。
    for i in range(count):
        img = ee.Image(l8_col_clip_list.get(i))
        print(f"""第{i+1}景影像处理中，未提示为研究区内无云""")
        l8_img_cloud_cal(img)
    
    #将选择的影像与少云影像集合合并
    selected_col = ee.ImageCollection(nice_list)
    
    return selected_col
    
    

In [12]:
result_col = l8_toa_select_by_cloud(l8_col,roi,0.1)

print(f"""共{result_col.size().getInfo()}景L8研究区无云影像""")
print(result_col.aggregate_array('system:index').getInfo())

27
第1景影像处理中，未提示为研究区内无云

            LC08_128031_20190703研究区云量为：0.00014164481664213913,全景云量为:0.0,
            被选中：True
第2景影像处理中，未提示为研究区内无云

            LC08_128031_20190804研究区云量为：0.5128451124408581,全景云量为:0.9255000305175781,
            被选中：False
第3景影像处理中，未提示为研究区内无云

            LC08_128031_20190820研究区云量为：0.4150532708839043,全景云量为:0.09159999847412109,
            被选中：False
第4景影像处理中，未提示为研究区内无云

            LC08_128031_20190905研究区云量为：0.033727334810878196,全景云量为:0.05449999809265137,
            被选中：True
第5景影像处理中，未提示为研究区内无云

            LC08_128031_20190921研究区云量为：0.20306787679150118,全景云量为:0.1365999984741211,
            被选中：False
第6景影像处理中，未提示为研究区内无云

            LC08_128031_20191007研究区云量为：0.2870847545515882,全景云量为:9.999999776482581e-05,
            被选中：False
第7景影像处理中，未提示为研究区内无云

            LC08_128031_20191023研究区云量为：0.9112145324255841,全景云量为:0.9025,
            被选中：False
第8景影像处理中，未提示为研究区内无云

            LC08_128032_20190703研究区云量为：5.425170621620179e-05,全景云量为:0.004199999868869781,
            被选中